In [1]:
#Run linear regressions to populate some of the null values in rankings/user and critics
import pickle
import numpy as np
import pandas as pd

#read in normalized dataframe
with open('testdata_df.pkl', 'r') as picklefile:
    testdata_df = pickle.load(picklefile)

#create dataframe querying from tomatoMeter, imdbRating and User rating where tomatoMeter is not N/A
train = testdata_df[['tomatoMeter', 'imdbRating', 'tomatoUserRating']].query('tomatoMeter != "N/A"')

#this will be our training data
trainTom = train['tomatoMeter']
trainImdb = train[['imdbRating']]    
train

,tomatoMeter,imdbRating,tomatoUserRating
0,100,8.3,3.7
1,48,6.9,3.2
2,17,6.6,3.1
3,52,5.7,3.3
4,48,5.9,3.0
5,86,8.2,3.9
6,65,6.3,3.1
7,25,5.6,2.7
8,52,5.7,2.6
9,78,7.2,3.4


In [2]:
from sklearn.linear_model import LinearRegression

#create instance of Linear Regression class
linreg = LinearRegression()

#fit the linear regression using training data
linreg.fit(trainImdb, trainTom)

# coefficients and intercept
linreg.coef_
linreg.intercept_

#define function that will use linear regression to predict missing value
def preddata(x):
    y = linreg.predict(x)
    z = y[0]
    if z > 100:
        return 100
    elif z < 0:
        return 0
    else:
        return z

#create new dataframe with the missing values
missingvals = testdata_df[['tomatoMeter', 'imdbRating']].query('tomatoMeter == "N/A"')

#pass through 'imdbRating' (our dependent variable) and apply linear regression to each value
#add predicted value of lin. regression to column with N/A values
missingvals['tomatoMeter'] = missingvals['imdbRating'].apply(preddata)

#concatanate the two dataframes we separated and sort it by index
frames = [missingvals, train]
populatednulls_df = pd.concat(frames)
populatednulls_df = populatednulls_df.sort_index()

In [3]:
#repeat procedure for tomatoUserRating
train2 = populatednulls_df[['tomatoMeter', 'imdbRating', 'tomatoUserRating']].query('tomatoUserRating != "N/A" and tomatoUserRating >= 0')

#training data
trainTom = train2['tomatoUserRating']
traindata = train2[['imdbRating', 'tomatoMeter']]  

linreg = LinearRegression()
linreg.fit(traindata, trainTom)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [4]:
def UserRating(x):
    if x > 0:
        return x
    else:
        return 'N/A'

#had empty values that were converted to N/A for purposes of querying dataframe
populatednulls_df['tomatoUserRating'] = populatednulls_df['tomatoUserRating'].apply(UserRating)

#query dataframe for rows where tomatoUserRating is 'N/A'
missingvals2 = populatednulls_df[['tomatoMeter', 'imdbRating', 'tomatoUserRating']].query('tomatoUserRating == "N/A"')

linregression = missingvals2[['imdbRating', 'tomatoMeter']]

#create list of predicted values of tomatoUserRating running linear regression
answers = linreg.predict(linregression)

#append list back into dataframe
missingvals2['tomatoUserRating'] = answers

In [5]:
from sklearn import preprocessing
from sklearn.preprocessing import Imputer

#concatanate two dataframes that we separated
frames = [missingvals2, train2]
populatednullsfinal_df = pd.concat(frames)
populatednullsfinal_df = populatednullsfinal_df.sort_index()


populatednullsfinal_df['imdbRating'] = preprocessing.scale(populatednullsfinal_df['imdbRating'])
populatednullsfinal_df['tomatoMeter'] = preprocessing.scale(populatednullsfinal_df['tomatoMeter'])
populatednullsfinal_df['tomatoUserRating'] = preprocessing.scale(populatednullsfinal_df['tomatoUserRating'])

//anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, _DataConversionWarning)


In [6]:
with open('populatednullsfinal_df.pkl', 'w') as picklefile:
    pickle.dump(populatednullsfinal_df, picklefile)

In [7]:
populatednullsfinal_df



,tomatoMeter,imdbRating,tomatoUserRating
0,1.469065,1.624699,0.719358
1,-0.483931,0.376212,-0.333909
2,-1.648217,0.108679,-0.544562
3,-0.333701,-0.693921,-0.123255
4,-0.483931,-0.515565,-0.755216
5,0.943258,1.535522,1.140665
6,0.154548,-0.158855,-0.544562
7,-1.347756,-0.783098,-1.387176
8,-0.333701,-0.693921,-1.597830
9,0.642798,0.643745,0.087398
